In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as grid_spec
from matplotlib.colors import ListedColormap
import itertools

import trueskill
from scipy.stats import norm
from sqlalchemy import select, insert, update, create_engine, Date, and_
from sqlalchemy.orm import Session, aliased

import data
from report_utils import ( 
    plot_match_qualities, 
    plot_ridge_skill_distributions, 
    plot_paired_skill_distributions, 
    plot_player_positions_bar,
    win_probability
    )
from common_queries import (
    get_player_skill_for_team,
    get_previous_matches
)


In [ ]:
sns.set_context("notebook")
sns.set_style("white")

In [ ]:
# TODO Better layout for this notebook
# Functions in notebook for:
#   - Previous lineup
#   - Position distribution per player
#   - Get best possible lineup (with best 4 of them?) 
engine = create_engine("sqlite:///../darts.db")
home_club = "Hannover 96"
away_club = "1.Dartclub Hildesheim e.V."
# away_club = "SC Drop Out Seelze e.V."
# away_club = "GVO Oldenburg"
# competition = "Verbandsliga 1"
competition = "DBH Bezirksliga 2"
ignore_home = ["Frank Fuhlrott", "Sven Rohde", "Oleksil Bondarenko", "Timo Trauslen", "Nils Oehlmann" ,"Andreas Schmidt"]
ignore_away = []

home_players = get_player_skill_for_team(engine, home_club, competition, ignore_home)
away_players = get_player_skill_for_team(engine, away_club, competition, ignore_away)


In [ ]:
f = plot_match_qualities(home_players, away_players)

In [ ]:
f = plot_paired_skill_distributions(home_players=home_players, away_players=away_players)


In [ ]:
f = plot_ridge_skill_distributions(home_players=home_players, away_players=away_players)


In [ ]:
fig = plot_player_positions_bar(away_players)
# TODO Fix: Get positions first, plot is separate function
# TODO Plot player positions as trajectory plot
# https://benalexkeen.com/parallel-coordinates-in-matplotlib/

In [ ]:
matches = get_previous_matches(home_players, away_players)
_ = [print(m) for m in matches]

In [ ]:
def compute_lineup_quality(my_lineup, opponent_lineup):
    ps = []
    for p1, p2 in zip(my_lineup, opponent_lineup):
        wp = win_probability([p1[2]], [p2[2]])
        ps.append(wp)
    hard = np.round(ps)
    return sum(ps), sum(hard)
        

def get_best_lineup(my_players, opponent_lineup, home=True):
    # TODO: Consider substitution player
    # TODO: Option to shift perms for away games, even more important :)
    # for a given opponent lineup
    # create all possible orderings for my_players
    # consider switching one player
    # compute win probability (soft and hard)
    # argmax over ratings to get best one
    my_players_perms = itertools.permutations(my_players)
    my_players_perms = [perm[:4] for perm in my_players_perms]

    perm_rank_soft = []
    perm_rank_hard = []
    for perm in my_players_perms:
        soft_rank, hard_rank = compute_lineup_quality(perm, opponent_lineup)
        perm_rank_soft.append(soft_rank)
        perm_rank_hard.append(hard_rank)

    # instead of shifting all permutations, just shift home downward
    if home:
        opponent_lineup.insert(0, opponent_lineup.pop()) # shift lineup from away team
    else:
        opponent_lineup.append(opponent_lineup.pop(0))
    # for each permutation we can substitute each player -> quadruples our permutations
    for i, perm in enumerate(my_players_perms):
        soft_rank, hard_rank = compute_lineup_quality(perm, opponent_lineup)
        perm_rank_soft[i] += soft_rank
        perm_rank_hard[i] += hard_rank


get_best_lineup(home_players, sorted(away_players, key=lambda p : p[2].mu, reverse=True)[:4])
